# Web Search with LlamaIndex

This example shows how to use the Python [LlamaIndex](https://docs.llamaindex.ai/en/stable/) library to run a text-generation request on open-source LLMs and embedding models using the OpenAI SDK, then augment that request using results from Google web search.


## Set up the RAG workflow environment

In [1]:
%%capture
!git clone https://github.com/VectorInstitute/rag_bootcamp /tmp/rag_bootcamp
!cp -ar /tmp/rag_bootcamp/document_search/* ./

# Install langchain and LlamaIndex packages, filtering out version numbers
!grep -Eo '^(langchain|llama)[^=]*' \
    /tmp/rag_bootcamp/envs/rag_dataloaders/requirements.txt > \
    /tmp/requirements.txt
!pip3 install -r /tmp/requirements.txt
!pip3 install faiss-cpu
!pip3 install googlesearch-python

In [ ]:
import os

# TODO: fill in the two following lines
os.environ["OPENAI_API_KEY"] = "..."
os.environ["OPENAI_BASE_URL"] = "https://.../v1"

#### Import libraries

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import faiss
import os
import requests

from bs4 import BeautifulSoup
from googlesearch import search

from langchain.text_splitter import RecursiveCharacterTextSplitter

from llama_index.core import Settings, VectorStoreIndex, StorageContext
from llama_index.core.llms import ChatMessage
from llama_index.core.node_parser import LangchainNodeParser
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.readers import StringIterableReader
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai_like import OpenAILike
from llama_index.vector_stores.faiss import FaissVectorStore

#### Load config files

In [5]:
GENERATOR_BASE_URL = os.environ["OPENAI_BASE_URL"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

#### Set up some helper functions

In [6]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.text for i, d in enumerate(docs)]
        )
    )

#### Choose LLM and embedding model

In [7]:
GENERATOR_MODEL_NAME = "Meta-Llama-3.1-8B-Instruct"

## Select one of the two options:
## - "all-MiniLM-L6-v2" (22M parameters)
## - "bge-base-en-v1.5" (110M parameters)

# EMBEDDING_MODEL_NAME = "bge-base-en-v1.5"
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"

## Start with a basic generation request without RAG augmentation

Let's start by asking Llama-3.1 a question about recent events that it doesn't know about, something that happened after it finished training. At the time I'm writing this notebook in November 2024, Llama3 doesn't know who won the last World Series of baseball.

*Who won the 2024 World Series of baseball?*

**The correct answer is the Los Angeles Dodgers won in October 2024.**

In [8]:
query = "Who won the 2024 World Series of baseball?"

## Now send the query to the open source model using KScope

In [9]:
llm = OpenAILike(
    model=GENERATOR_MODEL_NAME,
    is_chat_model=True,
    temperature=0,
    max_tokens=None,
    api_base=GENERATOR_BASE_URL,
    api_key=OPENAI_API_KEY,
    api_version="1"
)
message = [
    ChatMessage(
        role="user",
        content=query
    )
]
try:
    result = llm.chat(message)
    print(f"Result: \n\n{result}")
except Exception as err:
    if "Error code: 503" in err.message:
        print(f"The model {GENERATOR_MODEL_NAME} is not ready yet.")
    else:
        raise

Result: 

assistant: I don't have the ability to predict the future or know the outcome of future events, including the 2024 World Series. The 2024 World Series has not yet occurred, and I don't have any information about it. I can provide information about past World Series winners, though. Would you like to know more about that?


Llama-3.1 admits that it doesn't know the answer, since according to the model it's a future event.

Let's see how we can use RAG to augment our question with a Google web search and get the correct answer.

## Ingestion: Do a Google web search for the query and obtain the necessary information

Parse through all the websites returned by a Google search, break them up into smaller digestible chunks, then encode them as vector embeddings.

In [10]:
# Do a Google web search and store the results in a documents list
web_documents = []
for result_url in list(search(query))[:10]:
    try:
        response = requests.get(result_url)
        soup = BeautifulSoup(response.content, 'html.parser')
    except:
        # Skip if connection error
        continue

    web_documents.append(soup.get_text())

docs = StringIterableReader().load_data(texts=web_documents)
print(f"Number of source documents: {len(docs)}")

# Split the documents into smaller chunks
parser = LangchainNodeParser(RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=32))
chunks = parser.get_nodes_from_documents(docs)
print(f"Number of text chunks: {len(chunks)}")

Number of source documents: 9
Number of text chunks: 524


#### Define the embeddings model

In [ ]:
print(f"Setting up the embeddings model {EMBEDDING_MODEL_NAME} from {GENERATOR_BASE_URL}")
embeddings = OpenAIEmbedding(
    model_name=EMBEDDING_MODEL_NAME,
    embed_batch_size=128,
    api_base=GENERATOR_BASE_URL,
    api_key=OPENAI_API_KEY,
)

#### Set LLM and embedding model [recommended for LlamaIndex]

In [12]:
Settings.llm = llm
Settings.embed_model = embeddings

## Retrieval: Make the document chunks available via a retriever

The retriever will identify the document chunks that most closely match our original query. (This takes about 1-2 minutes)

In [13]:
def get_embed_model_dim(embed_model):
    embed_out = embed_model.get_text_embedding("Dummy Text")
    return len(embed_out)

faiss_dim = get_embed_model_dim(embeddings)
faiss_index = faiss.IndexFlatL2(faiss_dim)

vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(chunks, storage_context=storage_context)

In [14]:
retriever = index.as_retriever(similarity_top_k=5)

# Retrieve the most relevant context from the vector store based on the query
retrieved_docs = retriever.retrieve(query)

Let's see what results it found. Important to note, these results are in the order the retriever thought were the best matches.

In [15]:
pretty_print_docs(retrieved_docs)

Document 1:

The 2024 World Series was the championship series of Major League Baseball's (MLB) 2024 season. The 120th edition of the World Series, it was a best-of-seven playoff between the National League (NL) champion Los Angeles Dodgers and the American League (AL) champion New York Yankees. It was the Dodgers' first World Series appearance and win since 2020, and the Yankees' first World Series appearance since 2009. The series began on October 25 and ended on October 30 with the Dodgers winning in five
----------------------------------------------------------------------------------------------------
Document 2:

Blue Jays1994 - Not played (due to strike)1995 - Atlanta Braves1996 - New York Yankees1997 - Florida Marlins1998 - New York Yankees1999 - New York Yankees2000 - New York Yankees2001 - Arizona Diamondbacks2002 - Anaheim Angels2003 - Florida Marlins2004 - Boston Red Sox2005 - Chicago White Sox2006 - St. Louis Cardinals2007 - Boston Red Sox2008 - Philadelphia Phillies2009 

## Now send the query to the RAG pipeline

In [16]:
query_engine = RetrieverQueryEngine(retriever=retriever)
result = query_engine.query(query)
print(f"Result: \n\n{result}")

Result: 

The Los Angeles Dodgers won the championship in five games.


The model provides the correct answer based on the information from the web.